In [2]:
! pip show folium   # folium이 설치되어 있는지 확인
! pip install folium

In [8]:
import folium
map_osm = folium.Map(location=[33.4428621, 126.576983], zoom_start = 12)   # 위도, 경도 값 / zoom_start : 초기 화면의 크기 지정, 1이 가장 작은 값
map_osm.save('map1.html')   # 파일이 저장될 위치

In [9]:
map_osm

In [14]:
# 포리움은 기본적으로 'Open Street Map'을 기반으로 동작하지만 내부적으로는 'Stamen Terrain', 'Stamen Toner', 'Mapbox Bright', 'Mapbox Control room' tiles 형식을 내장하고 있음
map_osm = folium.Map(location=[33.4428621, 126.576983], zoom_start = 12, tiles = 'Stamen Terrain')
map_osm

In [15]:
# 마커와 팝업의 설정
   # 마커는 .Marker() 메서드를 이용, 생성한 포리움 객체에 추가(.add_to())하여 생성
map_osm = folium.Map(location=[33.4428621, 126.576983], zoom_start = 12)
folium.Marker([37.566345, 126.977893], popup = '서울특별시청').add_to(map_osm)
folium.Marker([37.5658859, 126.9754788], popup = '덕수궁').add_to(map_osm)
map_osm.save('./map.html')
map_osm

In [16]:
map_osm = folium.Map(location = [37.566345, 126.977893], zoom_start = 12)
folium.Marker([37.566345, 126.977893], popup = '서울특별시청', icon = folium.Icon(color='red', icon='info-sign')).add_to(map_osm)
folium.CircleMarker([37.5658859, 126.9754788], radius=100, color='#3186cc', fill_color='#3186cc', popup='덕수궁').add_to(map_osm)
map_osm.save('./map.html')
map_osm

In [64]:
import requests
import pandas as pd
import webbrowser

In [40]:
KAKAO_API_KEY = 'a5631c0f16e17a9e03697dc25d887d96'  
url = '''
      https://dapi.kakao.com/v2/local/search/keyword.json?query={0}&category_group_code=PO3
      '''.format('제주시청')
headers = {'Authorization': 'KakaoAK {0}'.format(KAKAO_API_KEY)}
res = requests.get(url, headers=headers)

In [41]:
res = res.json()

In [44]:
print(res['documents'][0]['address_name'],
res['documents'][0]['y'],
res['documents'][0]['x'],
res['documents'][0]['place_name'])

제주특별자치도 제주시 이도2동 1176-1 33.4995342412698 126.531171087129 제주시청


In [50]:
def net_get_geo_coder(query):
    KAKAO_API_KEY = 'a5631c0f16e17a9e03697dc25d887d96' 
    url = '''
          https://dapi.kakao.com/v2/local/search/keyword.json?query={0}&category_group_code=PO3
          '''.format(query)
    headers = {'Authorization': 'KakaoAK {0}'.format(KAKAO_API_KEY)}
    # API 쿼리 수행
    res = requests.get(url, headers=headers)
    res = res.json()
    # 데이터 리턴 : 주소, 위도, 경도, 이름 순으로 튜플로 리턴
    # return res['documents'][0]['address_name'], res['documents'][0]['y'], res['documents'][0]['x'], res['documents'][0]['place_name']
    try:
        return [float(res['documents'][0]['y']), float(res['documents'][0]['x'])]
    except:
        pass

In [59]:
geoData = []
geoData = net_get_geo_coder('제주특별자치도 제주시 천수로 9길')
geoData

In [60]:
map = folium.Map(location=[33.5116386887787, 126.538286857559], zoom_start=10)
filename = 'jeju.csv'
df = pd.read_csv(filename, encoding='CP949', header=0)
geoData = []

In [61]:
df.head()

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자,Unnamed: 8
0,제주시,일도1동,064-728-4412,제주특별자치도 제주시 성지로 71 (일도일동),인구 3146명 / 면적 0.33㎢ / 11통 61개반,지상 3층 / 연면적 585㎡,개소일 2000-12-20,2019-06-20,NaN
1,제주시,일도2동,064-728-4442,제주특별자치도 제주시 고마로 28 (일도이동),인구 38400명 / 면적2.18㎢ / 48통326반,지상 2층·지하 1층 / 연면적 1307.01,총면적 617.22㎡ / 개소일 2000-12-24,2019-06-20,NaN
2,제주시,이도1동,064-728-1530,제주특별자치도 제주시 중앙로25길 17 (이도1동),인구 7418명 / 면적 0.79㎢ / 16통 97개반,지상 2층 / 2층면적 205㎡,개소일 2000-12-28,2019-06-20,NaN
3,제주시,이도2동,064-755-2021,제주특별자치도 제주시 오복3길 8 (이도이동),인구 41428명 / 면적 5.4㎢ / 49통 323반,지하 1층·지상 3층 / 연면적 1223.52㎡,총면적 388.9㎡ / 개소일 2000-10-05,2019-06-20,NaN
4,제주시,삼도1동,064-728-4532,제주특별자치도 제주시 전농로 40 (삼도일동),인구 14281명 / 면적0.87㎢ / 18통122반,지하 1층·지상3층 / 연면적 2070.37㎡,총면적 456.55㎡,2019-06-20,NaN


In [62]:
# (일도일동) 제거
df['주소'] = df['주소'].str.split('(').str.get(0)   # .str : 문자열 관련 함수 / split() : 괄호 안에 있는 문자를 기준으로 나눔 / get(0) : 인덱스값 0(첫 번째)을 가져옴
# Unnamed: 8 컬럼 제거
df.drop(columns = 'Unnamed: 8')

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자
0,제주시,일도1동,064-728-4412,제주특별자치도 제주시 성지로 71,인구 3146명 / 면적 0.33㎢ / 11통 61개반,지상 3층 / 연면적 585㎡,개소일 2000-12-20,2019-06-20
1,제주시,일도2동,064-728-4442,제주특별자치도 제주시 고마로 28,인구 38400명 / 면적2.18㎢ / 48통326반,지상 2층·지하 1층 / 연면적 1307.01,총면적 617.22㎡ / 개소일 2000-12-24,2019-06-20
2,제주시,이도1동,064-728-1530,제주특별자치도 제주시 중앙로25길 17,인구 7418명 / 면적 0.79㎢ / 16통 97개반,지상 2층 / 2층면적 205㎡,개소일 2000-12-28,2019-06-20
3,제주시,이도2동,064-755-2021,제주특별자치도 제주시 오복3길 8,인구 41428명 / 면적 5.4㎢ / 49통 323반,지하 1층·지상 3층 / 연면적 1223.52㎡,총면적 388.9㎡ / 개소일 2000-10-05,2019-06-20
4,제주시,삼도1동,064-728-4532,제주특별자치도 제주시 전농로 40,인구 14281명 / 면적0.87㎢ / 18통122반,지하 1층·지상3층 / 연면적 2070.37㎡,총면적 456.55㎡,2019-06-20
5,제주시,삼도2동,064-728-4562,제주특별자치도 제주시 관덕로6길 15,인구 9431명 / 면적0.83㎢ / 19통109반,지하 1층·지상3층 / 연면적 814㎡,총면적 368.69㎡ / 개소일 2000-12-30,2019-06-20
6,제주시,용담1동,064-728-4592,제주특별자치도 제주시 용남3길 11,인구 8368명 / 면적0.61㎢ / 15통102반,지상 3층 / 대지면적 8825㎡ / 건축면적 428.55,연면적 112049㎡ / 개소일 2000-12-24,2019-06-20
7,제주시,용담2동,064-728-1535,제주특별자치도 제주시 흥운길 27,인구 16701명 / 면적 4.94㎢ / 23통 155반,지상 3층 지하 1층 / 연면적 1046.13㎡,총면적 449.55㎡ / 개관일 2000-12-30,2019-06-20
8,제주시,건입동,064-728-1536,제주특별자치도 제주시 만덕로 18,인구 10850명 / 면적2.5㎢ / 20통113반,지상 4층 / 연면적 1192.37㎡,총면적 323.92㎡ / 개소일 2000-12-27,2019-06-20
9,제주시,화북동,064-728-4682,제주특별자치도 제주시 진남로6길 17,인구 25559('17.5월 현재) / 면적8.28㎢ / 33통 205반,지하 1층·지상3층 / 연면적 2375.80㎡,연면적 54㎡ / 개소일 2000-12-27,2019-06-20


In [65]:
for index, row in df.iterrows():     # iterrows : row단위로 하나씩 리턴
    # print(index, row)
    geoData = net_get_geo_coder(row['주소'])
    print(geoData)
    if geoData != None:
        folium.Marker(geoData, popup = row['읍면동'], icon = folium.Icon(color='red')).add_to(map)
        
svFilename = 'jeju.html'
map.save(svFilename)
webbrowser.open(svFilename)

None
[33.5116386887787, 126.538286857559]
[33.5069402044009, 126.526976882749]
[33.4970469392912, 126.535290947586]
[33.5041154034239, 126.517362400655]
[33.5117364099539, 126.522210779319]
[33.50909779859489, 126.51326647018216]
[33.5114717601895, 126.511681792482]
[33.5150321080548, 126.531518045311]
None
[33.5218857941584, 126.585599365991]
[33.4917232671377, 126.594687244151]
[33.4763364526145, 126.545269320539]
[33.4951309224816, 126.511579851711]
[33.4881587279011, 126.496886364509]
[33.4830787815432, 126.477191625104]
[33.4928558848117, 126.432175190874]
[33.4997987984561, 126.458099340466]
[33.5029008833987, 126.468230562279]
[33.4127060174978, 126.261904781382]
None
[33.52252490132381, 126.85205144932898]
[33.5344085135487, 126.634132207107]
[33.3004273889195, 126.183868588586]
[33.9637161687759, 126.296120635111]
[33.5065180993215, 126.953381339676]
[33.2268205336185, 126.252132937392]
[33.2798775763008, 126.720665690663]
[33.4420840163751, 126.910957098008]
[33.2576271446112

True